# Redis hands-on 資料

https://gh.iiji.jp/iij-bootcamp/iij-bootcamp/tree/master/database/redis/

## 環境設定

In [1]:
# redis-py パッケージインストール
!pip install redis

     |████████████████████████████████| 71kB 2.0MB/s eta 0:00:01


In [2]:
# docker --link で渡ってきている環境変数を調べる
!env | grep 'REDIS_' | sort

REDIS_ENV_GOSU_VERSION=1.10
REDIS_ENV_REDIS_DOWNLOAD_SHA=2139009799d21d8ff94fc40b7f36ac46699b9e1254086299f8d3b223ca54a375
REDIS_ENV_REDIS_DOWNLOAD_URL=http://download.redis.io/releases/redis-5.0.5.tar.gz
REDIS_ENV_REDIS_VERSION=5.0.5
REDIS_NAME=/stoic_germain/redis
REDIS_PORT_6379_TCP_ADDR=172.17.0.2
REDIS_PORT_6379_TCP_PORT=6379
REDIS_PORT_6379_TCP_PROTO=tcp
REDIS_PORT_6379_TCP=tcp://172.17.0.2:6379
REDIS_PORT=tcp://172.17.0.2:6379


In [3]:
# 全体の基本設定
import sys
import os
import redis

HOST = os.environ['REDIS_PORT_6379_TCP_ADDR']
PORT = os.environ['REDIS_PORT_6379_TCP_PORT']
DB0 = 10
DB1 = 11
DB2 = 12

## 実際に Redis を使ってみる

### Redis に接続

In [4]:
r = redis.Redis(host=HOST, port=PORT, db=DB1, decode_responses=True)

### /etc/passwd の内容を Redis に入れてみる

In [5]:
# ファイルの内容を確認する
#!cat /etc/passwd

In [6]:
from pathlib import Path

file = Path('/etc/passwd')
with file.open() as f:
    for line in f:
        s = line.strip().split(':')
        r.hmset(s[0], { 'user': s[0], 'passwd': s[1], 'uid': s[2], 'gid': s[3], 'desc': s[4], 'home': s[5], 'shell': s[6] })

In [7]:
r.hgetall('gnats')

{'user': 'gnats',
 'passwd': 'x',
 'uid': '41',
 'gid': '41',
 'desc': 'Gnats Bug-Reporting System (admin)',
 'home': '/var/lib/gnats',
 'shell': '/usr/sbin/nologin'}

In [8]:
r.keys()

['list',
 'man',
 'lp',
 'games',
 'www-data',
 'sys',
 'proxy',
 'sync',
 'news',
 'irc',
 'root',
 'uucp',
 'nobody',
 'daemon',
 'backup',
 '_apt',
 'gnats',
 'mail',
 'jovyan',
 'bin']

### Message Queue を作ってみる

In [9]:
import time

QNAME = 'mqueue'

#r.unlink(QNAME)  # キューをクリアする

for i in range(5):
    r.lpush(QNAME, time.asctime())
    time.sleep(1)

r.lrange(QNAME, 0, -1)

print(r.rpop(QNAME))
print(r.rpop(QNAME))
r.lpush(QNAME, 'test1 ' + time.asctime())
time.sleep(1)

print(r.rpop(QNAME))
print(r.rpop(QNAME))
r.lpush(QNAME, 'test2 ' + time.asctime())

r.lrange(QNAME, 0, -1)


Fri Jul 12 05:04:49 2019
Fri Jul 12 05:04:50 2019
Fri Jul 12 05:04:51 2019
Fri Jul 12 05:04:52 2019


['test2 Fri Jul 12 05:04:55 2019',
 'test1 Fri Jul 12 05:04:54 2019',
 'Fri Jul 12 05:04:53 2019']

### expire の実験をする

In [10]:
import time

r1 = redis.Redis(host=HOST, port=PORT, db=DB2, decode_responses=True)
r1.flushall()

# 1 秒おきに 10 件のログを、expire 15秒で書き出す
print('output logs...', end='')
for i in range(10):
    r.set(time.asctime(), 'log %02d' % i, ex=15)
    print(i, end=' ')
    time.sleep(1)

# 1 秒おきに DB 内容を表示して、expire される様子を確かめる
print("\n\nchecking...")
time.sleep(1)
for i in range(14):
    print(r.keys('*'))
    time.sleep(1)

output logs...0 1 2 3 4 5 6 7 8 9 

checking...
['Fri Jul 12 05:05:20 2019', 'Fri Jul 12 05:05:19 2019', 'Fri Jul 12 05:05:11 2019', 'Fri Jul 12 05:05:12 2019', 'Fri Jul 12 05:05:15 2019', 'Fri Jul 12 05:05:16 2019', 'Fri Jul 12 05:05:17 2019', 'Fri Jul 12 05:05:18 2019', 'Fri Jul 12 05:05:14 2019', 'Fri Jul 12 05:05:13 2019']
['Fri Jul 12 05:05:20 2019', 'Fri Jul 12 05:05:19 2019', 'Fri Jul 12 05:05:11 2019', 'Fri Jul 12 05:05:12 2019', 'Fri Jul 12 05:05:15 2019', 'Fri Jul 12 05:05:16 2019', 'Fri Jul 12 05:05:17 2019', 'Fri Jul 12 05:05:18 2019', 'Fri Jul 12 05:05:14 2019', 'Fri Jul 12 05:05:13 2019']
['Fri Jul 12 05:05:20 2019', 'Fri Jul 12 05:05:19 2019', 'Fri Jul 12 05:05:11 2019', 'Fri Jul 12 05:05:12 2019', 'Fri Jul 12 05:05:15 2019', 'Fri Jul 12 05:05:16 2019', 'Fri Jul 12 05:05:17 2019', 'Fri Jul 12 05:05:18 2019', 'Fri Jul 12 05:05:14 2019', 'Fri Jul 12 05:05:13 2019']
['Fri Jul 12 05:05:20 2019', 'Fri Jul 12 05:05:19 2019', 'Fri Jul 12 05:05:11 2019', 'Fri Jul 12 05:05:12 201

## おまけ

In [ ]:
# DB を全クリアしたい場合は下記を実行する
#r.flushall()